In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# Medal Prediction

Individual medal prediction was based on MLP(Multi Layer Perceptron) method. The top 3 scoring athletes for each apparatus were each given a medal.

In [3]:
# read the file

data = pd.read_csv("gymnastics.csv")

# data preprocessing (handling missing values)
data_cleaned = data.dropna(subset=['Date', 'FirstName', 'Country', 'Rank', 'D_Score', 'E_Score', 'Score']).reset_index(drop=True)
data_cleaned['Penalty'].fillna(0, inplace=True)


data_cleaned= data_cleaned.replace({'Apparatus': {'VT1': 'VT', 'VT2': 'VT', 'UE' : 'UB'}})
rounds_to_combine = ['qual', 'AAqual', 'TeamQual']
data_cleaned['Round'] = data_cleaned['Round'].replace(rounds_to_combine, 'qual')

# 대Treat rows with the same 'Round', 'Apparatus', 'Score' for each competition and player as duplicates and keep only one
data_cleaned = data_cleaned.drop_duplicates(subset=['Competition', 'LastName', 'FirstName', 'Round', 'Apparatus']).reset_index(drop=True) #Score를 뺐음 (아마도 소숫점 때문에 오차가 나서 중복항 삭제가 잘 안 되는 듯)





In [4]:
data_cleaned['Medal'] = np.where(data_cleaned['Rank'].isin([1.0,2.0,3.0]), 1, 0)

In [5]:
# Extracting the feature and target columns
X = data_cleaned['Score'].values.reshape(-1, 1)  # Reshaping for sklearn
y = data_cleaned['Medal']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing the MLPClassifier
mlp = MLPClassifier(random_state=42)

# Fitting the model with the training data
mlp.fit(X_train, y_train)

# Predicting the Test set results
y_pred = mlp.predict(X_test)

# Evaluating the predictions
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

accuracy, report

(0.9062740076824584,
 '              precision    recall  f1-score   support\n\n           0       0.92      0.98      0.95      3465\n           1       0.67      0.33      0.45       440\n\n    accuracy                           0.91      3905\n   macro avg       0.79      0.66      0.70      3905\nweighted avg       0.89      0.91      0.89      3905\n')

In [6]:
USA_gym = pd.read_csv('USA_gym.csv')

# Woman medal prediction

In [46]:
# select players based on the prediction score
best_team_w = USA_gym[USA_gym['Athlete'].isin(['BILES Simone', 'CAREY Jade', 'JONES Shilese', 'LEE Sunisa','LINCOLN Kaliya'])]

score_pred_w = np.array(best_team_w["Score_Prediction"])


In [52]:
score_pred_w.shape

(33,)

In [53]:
score_pred_w = score_pred_w.reshape((33, 1))

In [54]:
# model fitting
medal_prediction_w = mlp.predict(score_pred_w)

In [55]:
best_team_w["Medal_Prediction"]=medal_prediction_w

C:\Users\sbjbo\AppData\Local\Temp\ipykernel_39752\557048952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_team_w["Medal_Prediction"]=medal_prediction_w


In [56]:
best_team_w

Unnamed: 0 FirstName LastName    Round Apparatus Gender Country  \
417        3900      Jade    CAREY  AAfinal        BB      w     USA   
418        3901      Jade    CAREY    final        BB      w     USA   
419        3903      Jade    CAREY  AAfinal        FX      w     USA   
420        3904      Jade    CAREY    final        FX      w     USA   
421        3906      Jade    CAREY  AAfinal        UB      w     USA   
422        3908      Jade    CAREY  AAfinal        VT      w     USA   
423        3909      Jade    CAREY    final        VT      w     USA   
495        4701    Kaliya  LINCOLN  AAfinal        BB      w     USA   
496        4702    Kaliya  LINCOLN    final        BB      w     USA   
497        4704    Kaliya  LINCOLN  AAfinal        FX      w     USA   
498        4705    Kaliya  LINCOLN    final        FX      w     USA   
499        4707    Kaliya  LINCOLN  AAfinal        UB      w     USA   
778        8257   Shilese    JONES  AAfinal        BB      w     USA   
779        8258   Shilese    JONES    final        BB      w     USA   
780        8260   Shilese    JONES  AAfinal        FX      w     USA   
781        8261   Shilese    JONES    final        FX      w     USA   
782        8263   Shilese    JONES  AAfinal        UB      w     USA   
783        8264   Shilese    JONES    final        UB      w     USA   
784        8266   Shilese    JONES  AAfinal        VT      w     USA   
785        8326    Simone    BILES  AAfinal        BB      w     USA   
786        8327    Simone    BILES    final        BB      w     USA   
787        8329    Simone    BILES  AAfinal        FX      w     USA   
788        8330    Simone    BILES    final        FX      w     USA   
789        8332    Simone    BILES  AAfinal        UB      w     USA   
790        8333    Simone    BILES    final        UB      w     USA   
791        8335    Simone    BILES  AAfinal        VT      w     USA   
792        8336    Simone    BILES    final        VT      w     USA   
810        8521    Sunisa      LEE  AAfinal        BB      w     USA   
811        8522    Sunisa      LEE    final        BB      w     USA   
812        8524    Sunisa      LEE  AAfinal        FX      w     USA   
813        8526    Sunisa      LEE  AAfinal        UB      w     USA   
814        8527    Sunisa      LEE    final        UB      w     USA   
815        8529    Sunisa      LEE  AAfinal        VT      w     USA   

           Date                                        Competition  \
417  2023-08-04                        2023 Core Hydration Classic   
418  2022-09-24                     2022 Paris World Challenge Cup   
419  2022-11-06  2022 51st FIG Artistic Gymnastics World Champi...   
420  2022-11-06  2022 51st FIG Artistic Gymnastics World Champi...   
421  2022-11-06  2022 51st FIG Artistic Gymnastics World Champi...   
422  2022-11-06  2022 51st FIG Artistic Gymnastics World Champi...   
423  2022-11-06  2022 51st FIG Artistic Gymnastics World Champi...   
495  2023-08-04                        2023 Core Hydration Classic   
496  2023-10-21               SANTIAGO 2023 XIX Pan American Games   
497  2023-08-04                        2023 Core Hydration Classic   
498  2023-10-21               SANTIAGO 2023 XIX Pan American Games   
499  2023-02-24                                    2023 Winter Cup   
778  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
779  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
780  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
781  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
782  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
783  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
784  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
785  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
786  2023-10-08  2023 52nd FIG Artistic Gymnastics World Champi...   
787  2023-10-08 

In [59]:
best_team_w = best_team_w[['Athlete', 'Round', 'Apparatus','Score_Prediction','Medal_Prediction']]

In [75]:
best_team_w

Athlete    Round Apparatus  Score_Prediction  Medal_Prediction
417      CAREY Jade  AAfinal        BB         13.513639                 0
418      CAREY Jade    final        BB         12.898303                 0
419      CAREY Jade  AAfinal        FX         14.020990                 0
420      CAREY Jade    final        FX         13.946219                 0
421      CAREY Jade  AAfinal        UB         13.112485                 0
422      CAREY Jade  AAfinal        VT         14.438005                 0
423      CAREY Jade    final        VT         14.550428                 1
495  LINCOLN Kaliya  AAfinal        BB         13.351761                 0
496  LINCOLN Kaliya    final        BB         13.200818                 0
497  LINCOLN Kaliya  AAfinal        FX         14.178653                 0
498  LINCOLN Kaliya    final        FX         14.280256                 0
499  LINCOLN Kaliya  AAfinal        UB         13.193085                 0
778   JONES Shilese  AAfinal        BB         13.731583                 0
779   JONES Shilese    final        BB         13.446031                 0
780   JONES Shilese  AAfinal        FX         13.636383                 0
781   JONES Shilese    final        FX         13.785485                 0
782   JONES Shilese  AAfinal        UB         14.903877                 1
783   JONES Shilese    final        UB         14.480194                 1
784   JONES Shilese  AAfinal        VT         14.176430                 0
785    BILES Simone  AAfinal        BB         14.419993                 0
786    BILES Simone    final        BB         14.193394                 0
787    BILES Simone  AAfinal        FX         14.892567                 1
788    BILES Simone    final        FX         14.545826                 1
789    BILES Simone  AAfinal        UB         14.033108                 0
790    BILES Simone    final        UB         14.317711                 0
791    BILES Simone  AAfinal        VT         15.420900                 1
792    BILES Simone    final        VT         13.999699                 0
810      LEE Sunisa  AAfinal        BB         13.869656                 0
811      LEE Sunisa    final        BB         13.841568                 0
812      LEE Sunisa  AAfinal        FX         13.371685                 0
813      LEE Sunisa  AAfinal        UB         14.652035                 1
814      LEE Sunisa    final        UB         14.872469                 1
815      LEE Sunisa  AAfinal        VT         14.562347                 1

# Man medal prediction

In [64]:
# select players based on the prediction score
best_team_m = USA_gym[USA_gym['Athlete'].isin(['HONG Asher', 'MALONE Brody', 'WISKUS Shane', 'YOUNG Khoi','PHILLIPS Curran'])]

score_pred_m = np.array(best_team_m["Score_Predcition"])

In [65]:
score_pred_m.shape

(39,)

In [66]:
score_pred_m = score_pred_m.reshape((39, 1))

In [67]:
# model fitting
medal_prediction_m = mlp.predict(score_pred_m)

In [68]:
best_team_m["Medal_Prediction"]=medal_prediction_m

C:\Users\sbjbo\AppData\Local\Temp\ipykernel_39752\14461383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_team_m["Medal_Prediction"]=medal_prediction_m


In [70]:
best_team_m = best_team_m[['Athlete', 'Round', 'Apparatus','Score_Prediction','Medal_Prediction']]

In [73]:
best_team_m

Athlete    Round Apparatus  Score_Prediction  Medal_Prediction
54        HONG Asher  AAfinal        FX         14.062750                 0
55        HONG Asher  AAfinal        HB         12.530790                 0
56        HONG Asher  AAfinal        PB         14.290291                 0
57        HONG Asher    final        PB         14.041542                 0
58        HONG Asher  AAfinal        PH         12.706124                 0
59        HONG Asher    final        PH         12.962471                 0
60        HONG Asher  AAfinal        SR         14.072449                 0
61        HONG Asher    final        SR         13.879915                 0
62        HONG Asher  AAfinal        VT         14.178295                 0
63        HONG Asher    final        VT         14.869057                 1
106     MALONE Brody  AAfinal        FX         13.443098                 0
107     MALONE Brody  AAfinal        HB         13.307528                 0
108     MALONE Brody    final        HB         14.337938                 0
109     MALONE Brody  AAfinal        PB         14.031466                 0
110     MALONE Brody    final        PB         14.203604                 0
111     MALONE Brody  AAfinal        PH         13.676336                 0
112     MALONE Brody    final        PH         13.666485                 0
113     MALONE Brody  AAfinal        SR         13.662375                 0
114     MALONE Brody    final        SR         13.950905                 0
115     MALONE Brody  AAfinal        VT         14.412266                 0
215  PHILLIPS Curran  AAfinal        HB         14.172194                 0
216  PHILLIPS Curran    final        HB         13.992470                 0
217  PHILLIPS Curran  AAfinal        PB         14.299543                 0
218  PHILLIPS Curran    final        PB         15.132078                 1
219  PHILLIPS Curran  AAfinal        VT         14.625566                 1
544       YOUNG Khoi  AAfinal        FX         14.447371                 0
545       YOUNG Khoi  AAfinal        HB         12.322660                 0
546       YOUNG Khoi  AAfinal        PB         14.746084                 1
547       YOUNG Khoi  AAfinal        PH         13.602676                 0
548       YOUNG Khoi    final        PH         13.674330                 0
549       YOUNG Khoi  AAfinal        SR         13.290311                 0
550       YOUNG Khoi  AAfinal        VT         14.972307                 1
551       YOUNG Khoi    final        VT         15.114654                 1
772     WISKUS Shane  AAfinal        FX         14.017761                 0
773     WISKUS Shane  AAfinal        HB         14.041380                 0
774     WISKUS Shane  AAfinal        PB         14.633082                 1
775     WISKUS Shane  AAfinal        PH         13.508825                 0
776     WISKUS Shane  AAfinal        SR         14.066788                 0
777     WISKUS Shane  AAfinal        VT         14.409405                 0